# Conditional GAN (cGAN) - Cats vs Dogs Image Generation
## Project Overview
This notebook implements a Conditional Generative Adversarial Network (cGAN) to generate images of cats or dogs conditioned on class labels.

- **Label 0**: Cat
- **Label 1**: Dog

The model learns to generate realistic cat and dog images based on the specified label.


In [ ]:
# Import required libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Configuration
IMG_SIZE = 64
NOISE_DIM = 100
NUM_CLASSES = 2
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.0002

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## Part 1: Load and Prepare Cats vs Dogs Dataset

We will load the Cats vs Dogs dataset using `tensorflow_datasets` with split information enabled.
The dataset contains over 23,000 labeled images:
- Label 0 = Cat
- Label 1 = Dog

In [ ]:
# Load the Cats vs Dogs dataset with info
print("Loading Cats vs Dogs dataset...")
(train_images, train_labels), (test_images, test_labels), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

print(f"Dataset info:\n{info}")
print(f"\nTraining samples: {len(list(train_images))}")
print(f"Testing samples: {len(list(test_images))}")


## Part 2: Preprocess Images and Labels

We will:
1. Resize all images to 64×64 pixels
2. Normalize pixel values to [-1, 1] range
3. Cast labels to integer format
4. Create training batches with labels

In [ ]:
def preprocess_image(image, label):
    """
    Preprocess image and label:
    - Resize image to IMG_SIZE x IMG_SIZE
    - Normalize pixel values to [-1, 1] range
    - Cast label to int32
    """
    # Convert image to float32
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Resize to 64x64
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    
    # Normalize to [-1, 1] range (from [0, 1])
    image = (image * 2.0) - 1.0
    
    # Cast label to int32
    label = tf.cast(label, tf.int32)
    
    return image, label

# Apply preprocessing and batching to training dataset
train_dataset = train_images.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.cache().shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Apply preprocessing and batching to test dataset
test_dataset = test_images.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"Training dataset: {train_dataset}")
print(f"Test dataset: {test_dataset}")

# Visualize a few preprocessed samples
sample_images, sample_labels = next(iter(train_dataset.take(1)))
print(f"\nSample batch shape: {sample_images.shape}")
print(f"Sample labels: {sample_labels.numpy()}")
print(f"Image value range: [{sample_images.numpy().min():.2f}, {sample_images.numpy().max():.2f}]")


## Part 3: Build Generator Network

The generator:
- Takes a noise vector and class label as input
- Embeds the label and concatenates with noise
- Uses Conv2DTranspose layers to upsample to 64×64×3 image
- Outputs image with tanh activation (values in [-1, 1])

In [ ]:
def build_generator():
    """
    Build the generator network that takes noise and class label and generates 64x64x3 images.
    """
    # Inputs
    noise_input = tf.keras.layers.Input(shape=(NOISE_DIM,), name='noise')
    label_input = tf.keras.layers.Input(shape=(), dtype='int32', name='label')
    
    # Label embedding: Convert label to embedding vector
    label_emb = tf.keras.layers.Embedding(NUM_CLASSES, 50)(label_input)
    label_emb = tf.keras.layers.Flatten()(label_emb)
    
    # Concatenate noise and label embedding
    x = tf.keras.layers.Concatenate()([noise_input, label_emb])
    
    # Dense layer to project to initial spatial dimensions (8x8x256)
    x = tf.keras.layers.Dense(8 * 8 * 256, use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    
    # Reshape to 8x8x256
    x = tf.keras.layers.Reshape((8, 8, 256))(x)
    
    # Conv2DTranspose layer 1: 8x8 -> 16x16
    x = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    
    # Conv2DTranspose layer 2: 16x16 -> 32x32
    x = tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    
    # Conv2DTranspose layer 3: 32x32 -> 64x64
    x = tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')(x)
    
    # Create model
    generator = tf.keras.Model([noise_input, label_input], x, name='generator')
    return generator

# Build generator
generator = build_generator()
generator.summary()


## Part 4: Build Discriminator Network

The discriminator:
- Takes a 64×64×3 image and class label as input
- Embeds the label and concatenates with image
- Uses Conv2D layers to downsampl and classify
- Outputs a single value (real=1, fake=0)

In [ ]:
def build_discriminator():
    """
    Build the discriminator network that takes 64x64x3 images and class labels
    and outputs a single value indicating real or fake.
    """
    # Inputs
    image_input = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='image')
    label_input = tf.keras.layers.Input(shape=(), dtype='int32', name='label')
    
    # Label embedding: Convert label to spatial map
    label_emb = tf.keras.layers.Embedding(NUM_CLASSES, 50)(label_input)
    label_emb = tf.keras.layers.Flatten()(label_emb)
    
    # Expand label to spatial dimensions and concatenate with image
    label_map = tf.keras.layers.Dense(IMG_SIZE * IMG_SIZE * 1)(label_emb)
    label_map = tf.keras.layers.Reshape((IMG_SIZE, IMG_SIZE, 1))(label_map)
    
    x = tf.keras.layers.Concatenate()([image_input, label_map])
    
    # Conv2D layer 1: 64x64 -> 32x32
    x = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    # Conv2D layer 2: 32x32 -> 16x16
    x = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    # Conv2D layer 3: 16x16 -> 8x8
    x = tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    # Flatten and output
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1)(x)
    
    # Create model
    discriminator = tf.keras.Model([image_input, label_input], x, name='discriminator')
    return discriminator

# Build discriminator
discriminator = build_discriminator()
discriminator.summary()


## Part 5: Define Loss Functions and Optimizers

We use:
- **Binary Cross-Entropy Loss**: For both generator and discriminator
- **Generator Loss**: Encourages discriminator to classify generated images as real
- **Discriminator Loss**: Distinguishes between real and fake images
- **Adam Optimizer**: For both networks with learning rate 0.0002 and beta_1=0.5

In [ ]:
# Define loss functions
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    """
    Generator loss: We want the discriminator to classify generated images as real (label=1).
    """
    return cross_entropy_loss(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    """
    Discriminator loss: Distinguish between real (label=1) and fake (label=0) images.
    """
    real_loss = cross_entropy_loss(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy_loss(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, beta_1=0.5)

print("Loss functions and optimizers defined successfully!")


## Part 6: Create Training Loop

The training process:
1. For each batch: Generate fake images from noise and labels
2. Discriminator: Classify real and fake images
3. Compute losses and apply gradients alternately
4. Use @tf.function for optimization
5. Train for 10+ epochs

In [ ]:
# Store losses for visualization
train_gen_losses = []
train_disc_losses = []

@tf.function
def train_step(images, labels):
    """
    Single training step: Update generator and discriminator alternately.
    
    Args:
        images: Batch of real images
        labels: Batch of labels
        
    Returns:
        Generator loss and discriminator loss
    """
    # Generate random noise for generator input
    noise = tf.random.normal([tf.shape(images)[0], NOISE_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator([noise, labels], training=True)
        
        # Discriminator predictions
        real_output = discriminator([images, labels], training=True)
        fake_output = discriminator([generated_images, labels], training=True)
        
        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # Apply gradients
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

# Training loop
print("Starting training...")
for epoch in range(EPOCHS):
    epoch_gen_loss = 0.0
    epoch_disc_loss = 0.0
    num_batches = 0
    
    for images, labels in train_dataset:
        gen_loss, disc_loss = train_step(images, labels)
        epoch_gen_loss += gen_loss
        epoch_disc_loss += disc_loss
        num_batches += 1
    
    # Calculate average losses for the epoch
    avg_gen_loss = epoch_gen_loss / num_batches
    avg_disc_loss = epoch_disc_loss / num_batches
    
    train_gen_losses.append(avg_gen_loss)
    train_disc_losses.append(avg_disc_loss)
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}")

print("Training completed!")


In [ ]:
# Plot training losses
plt.figure(figsize=(10, 5))
plt.plot(train_gen_losses, label='Generator Loss')
plt.plot(train_disc_losses, label='Discriminator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


## Part 7: Generate and Visualize Conditional Images

Generate images conditioned on class labels and display with titles.

In [ ]:
def generate_and_display_images(num_examples_per_class=4):
    """
    Generate images conditioned on labels and display with titles.
    
    Args:
        num_examples_per_class: Number of images to generate for each class
    """
    # Label names
    label_names = {0: 'Cat', 1: 'Dog'}
    
    # Create figure for generated images
    fig, axes = plt.subplots(2, num_examples_per_class, figsize=(15, 6))
    fig.suptitle('Generated Images Conditioned on Class Labels', fontsize=16)
    
    for class_label in range(NUM_CLASSES):
        # Generate random noise
        noise = tf.random.normal([num_examples_per_class, NOISE_DIM])
        
        # Create labels array
        labels = np.full((num_examples_per_class,), class_label, dtype=np.int32)
        
        # Generate images
        generated_images = generator([noise, labels], training=False)
        
        # Normalize images from [-1, 1] to [0, 1] for display
        generated_images = (generated_images + 1.0) / 2.0
        
        # Display images
        for i in range(num_examples_per_class):
            ax = axes[class_label, i]
            ax.imshow(generated_images[i].numpy())
            ax.set_title(f'{label_names[class_label]}')
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Generate and display conditional images
generate_and_display_images(num_examples_per_class=4)


In [ ]:
# Additional visualization: Generate more samples and create a larger grid
def generate_large_sample_grid(num_per_class=8):
    """Generate a larger grid of images for visual evaluation."""
    label_names = {0: 'Cat', 1: 'Dog'}
    
    fig = plt.figure(figsize=(16, 8))
    fig.suptitle('cGAN Generated Image Grid - Label-Conditioned Samples', fontsize=16)
    
    image_count = 1
    for class_label in range(NUM_CLASSES):
        # Generate noise and labels
        noise = tf.random.normal([num_per_class, NOISE_DIM])
        labels = np.full((num_per_class,), class_label, dtype=np.int32)
        
        # Generate images
        generated_images = generator([noise, labels], training=False)
        generated_images = (generated_images + 1.0) / 2.0  # Normalize to [0, 1]
        
        # Display in grid
        for i in range(num_per_class):
            ax = plt.subplot(2, num_per_class, image_count)
            ax.imshow(generated_images[i].numpy())
            ax.set_title(f'{label_names[class_label]} #{i+1}', fontsize=10)
            ax.axis('off')
            image_count += 1
    
    plt.tight_layout()
    plt.show()

# Generate large sample grid
generate_large_sample_grid(num_per_class=8)


In [ ]:
# Summary of the cGAN Model
print("=" * 60)
print("CONDITIONAL GAN (cGAN) - TRAINING SUMMARY")
print("=" * 60)
print(f"Total Epochs Trained: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Noise Dimension: {NOISE_DIM}")
print(f"Image Size: {IMG_SIZE} x {IMG_SIZE}")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"\nFinal Generator Loss: {train_gen_losses[-1]:.4f}")
print(f"Final Discriminator Loss: {train_disc_losses[-1]:.4f}")
print("\nModel Architectures:")
print("- Generator: Accepts noise + label → Generates 64x64x3 images")
print("- Discriminator: Accepts image + label → Outputs real/fake prediction")
print("\nKey Features:")
print("- Label embedding and concatenation")
print("- Batch normalization for stable training")
print("- LeakyReLU activation functions")
print("- Binary cross-entropy loss")
print("- Adam optimizer with lr=0.0002, beta_1=0.5")
print("=" * 60)
